In [ ]:
!pip install llmlingua datasets qa_metrics ijson transformers==4.51.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 120.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import time
from torch.optim import Adam
from torch.distributions import Categorical
import datasets
from transformers import AutoTokenizer , AutoModel , pipeline
from tqdm import tqdm
import random
from qa_metrics.em import em_match
import ijson
from datasets import Dataset , concatenate_datasets , DatasetDict, load_from_disk
from decimal import Decimal
from qa_metrics.f1 import f1_score_with_precision_recall
import wandb
#wandb.login()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [ ]:
!unzip -q tqa-dataset.zip -d /content/tqa-dataset/

In [ ]:
tqa_dataset = Dataset.from_json(r"/content/test_tqa_base.json")
nqa_dataset = Dataset.from_json("/content/test_nqa_base.json")
hqa_dataset = Dataset.from_json("/content/test_hotpotqa_base.json")
wiki_dataset = Dataset.from_json("/content/test_2wiki_base.json")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
filter_tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

def preprocess(example):
    limit_context = 1
    question = example["question"]
    ctxs = example["ctxs"]
    context = "\n".join(ctx["text"] for ctx in ctxs[:limit_context])

    input_ids = filter_tokenizer(
        question,
        context,
        truncation="only_second",
        max_length=1024,
        padding="max_length",
        return_tensors = "pt",
        return_special_tokens_mask = True
    )
    mask = input_ids["special_tokens_mask"][0]  # shape: (max_length,)

    # Find indices of 1s
    one_indices = (mask == 1).nonzero(as_tuple=True)[0]

    # Make sure there are at least two 1s
    if len(one_indices) >= 2:
        start, end = one_indices[0], one_indices[1]
        mask[start:end + 1] = 1  # Set all values between (and including) first two 1s to 1
    mask = (mask == 0).int()

    # Optionally: update back to batch tensor
    input_ids["special_tokens_mask"][0] = mask
    input_ids_filtered = input_ids["input_ids"][0][mask == 1]  # Apply the mask to filter input_ids

    labels = [1 if ctx["has_answer"] else 0 for ctx in ctxs[:limit_context]]

    return {
        "input_ids": input_ids["input_ids"][0],
        "attention_mask" : input_ids["attention_mask"][0],
        "tokens_mask" : input_ids["special_tokens_mask"][0],
        "answer": example["answers"],
        "context" : context,
        "question" : question,
        "label" : any(labels)
    }

tqa_dataset = tqa_dataset.map(preprocess, remove_columns=tqa_dataset.column_names)
nqa_dataset = nqa_dataset.map(preprocess, remove_columns=nqa_dataset.column_names)
hqa_dataset = hqa_dataset.map(preprocess, remove_columns=hqa_dataset.column_names)
wiki_dataset = wiki_dataset.map(preprocess, remove_columns=wiki_dataset.column_names)
test_dataset = datasets.concatenate_datasets([tqa_dataset,hqa_dataset,nqa_dataset,wiki_dataset])
test_dataset.save_to_disk('/content/test-dataset')

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

PCRL

In [ ]:
class BasePolicy(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
        self.layer1 = nn.Linear(base_model.config.hidden_size, 4096)

class ActorNet(BasePolicy):
    def __init__(self, base_model, act_dim):
        super().__init__(base_model)
        self.layer2 = nn.Linear(4096, 4096)
        self.layer3 = nn.Linear(4096, act_dim)

    def forward(self, input_ids, attention_mask):
        features = self.base_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        x = F.relu(self.layer1(features))
        x = F.relu(self.layer2(x))
        logits = self.layer3(x)  # (batch, seq_len, act_dim)
        return logits

base_model = AutoModel.from_pretrained("answerdotai/ModernBERT-base")
base_model.eval()
for param in base_model.parameters():
    param.requires_grad = False
model = ActorNet(base_model,act_dim=2)

model.load_state_dict(torch.load("actor_best_new.pt", weights_only=True,map_location="cuda"))
model = model.to("cuda")

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

In [ ]:
next(model.parameters()).is_cuda

True

In [ ]:
compress_output = []
start_time = time.time()
#wandb.init(project="pcrl_exps", name=("PCRL compress final"))
for i in tqdm(range(0, len(test_dataset),16), desc="🧠 Compressing answers"):
    sub_batch = test_dataset[i:i+16]
    token_mask = torch.LongTensor(sub_batch["tokens_mask"]).to("cuda")
    input_ids = torch.LongTensor(sub_batch["input_ids"]).to("cuda")
    attention_mask = torch.LongTensor(sub_batch["attention_mask"]).to("cuda")
    sub_batch_output = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
    )

    # wandb.log({
    #         "VRAM Allocated (MB)": torch.cuda.memory_allocated() / (1024**2),
    #         "VRAM Reserved (MB)": torch.cuda.memory_reserved() / (1024**2)
    # })
    dist = Categorical(logits=sub_batch_output)
    samples = dist.sample()
    masked_samples = torch.where(token_mask.bool(), samples, torch.tensor(0, device=samples.device))

    for i in range(masked_samples.size(0)):  # batch loop
        act = masked_samples[i]
        mask = token_mask[i]
        ids = input_ids[i]
        # Filter only positions where tokens_mask == 1
        act = act[mask == 1]
        ids = ids[mask == 1]
        kept_ids = ids[act == 0]

        context = filter_tokenizer.decode(kept_ids)

        comp_rate = 1 - (torch.numel(kept_ids) / torch.numel(ids))

        compress_output.append({"context" : context,
                                "question" : sub_batch["question"][i],
                                "answer": sub_batch["answer"][i],
                                "compress_rate" : comp_rate,
                                "pre_label" : sub_batch["label"][i],
                                "post_label" : any([ans in context.lower() for ans in sub_batch["answer"][i]]),
                                })

total_time = time.time() - start_time
print("\n===============\nTotal Time:",total_time,"\n===============\n")
#wandb.finish()
pcrl_dataset = Dataset.from_list(compress_output)
pcrl_dataset.save_to_disk('/content/pcrl-dataset')

🧠 Compressing answers:   0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0511 04:10:04.349000 4149 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode
🧠 Compressing answers: 100%|██████████| 500/500 [07:02<00:00,  1.18it/s]


Total Time: 422.1375238895416 



Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
sum(pcrl_dataset["post_label"])

1641

In [ ]:
compress_rate = sum(pcrl_dataset["compress_rate"])/len(pcrl_dataset["compress_rate"])

0.9484075954624412

llmlingua-1

In [ ]:
from llmlingua import PromptCompressor
llm_lingua = PromptCompressor()
compress_output = []
start_time = time.time()
#wandb.init(project="pcrl_exps", name=("llmlingua-1 compress final"))

for i in tqdm(range(len(test_dataset)), desc="🧠 Compressing answers"):
    llmlingua_context = llm_lingua.compress_prompt(
        test_dataset["context"][i],
        rate=1-compress_rate,
        question=test_dataset["question"][i],
    )

    # wandb.log({
    #         "VRAM Allocated (MB)": torch.cuda.memory_allocated() / (1024**2),
    #         "VRAM Reserved (MB)": torch.cuda.memory_reserved() / (1024**2)
    # })

    comp_rate = 1 - (llmlingua_context["compressed_tokens"] / llmlingua_context["origin_tokens"])

    compress_output.append({"context":llmlingua_context['compressed_prompt'].removesuffix("\n\n"+test_dataset["question"][i]),
                            "question":test_dataset["question"][i],
                            "compress_rate" : comp_rate,
                            "pre_label" : test_dataset["label"][i],
                            "post_label" : any([ans in llmlingua_context['compressed_prompt'].lower() for ans in test_dataset["answer"][i]]),
                            })

total_time = time.time() - start_time
print("\n===============\nTotal Time:",total_time,"\n===============\n")
#wandb.finish()
lingua_dataset = Dataset.from_list(compress_output)
lingua_dataset.save_to_disk('llmlingua1-dataset')

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

🧠 Compressing answers: 100%|██████████| 8000/8000 [26:07<00:00,  5.10it/s]



Total Time: 1567.096881389618 



Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

llmlingua-2

In [ ]:
from llmlingua import PromptCompressor
compress_output = []
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
    use_llmlingua2=True,
)
start_time = time.time()
#wandb.init(project="pcrl_exps", name=("llmlingua-2 compress final"))


for i in tqdm(range(len(test_dataset)), desc="🧠 Compressing answers"):
    llmlingua_context = llm_lingua.compress_prompt(
        test_dataset["context"][i],
        rate=1-compress_rate,
        drop_consecutive=True,
    )

    # wandb.log({
    #         "VRAM Allocated (MB)": torch.cuda.memory_allocated() / (1024**2),
    #         "VRAM Reserved (MB)": torch.cuda.memory_reserved() / (1024**2)
    # })

    comp_rate = 1 - (llmlingua_context["compressed_tokens"] / llmlingua_context["origin_tokens"])
    compress_output.append({"context":llmlingua_context['compressed_prompt'],
                            "question":test_dataset["question"][i],
                            "compress_rate" : comp_rate,
                            "pre_label" : test_dataset["label"][i],
                            "post_label" : any([ans in llmlingua_context['compressed_prompt'].lower() for ans in test_dataset["answer"][i]]),
                            })

total_time = time.time() - start_time
print("\n===============\nTotal Time:",total_time,"\n===============\n")
#wandb.finish()
lingua_dataset = Dataset.from_list(compress_output)
lingua_dataset.save_to_disk('llmlingua2-dataset')

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

🧠 Compressing answers: 100%|██████████| 8000/8000 [18:47<00:00,  7.10it/s]


Total Time: 1127.2256450653076 



Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
sum(lingua_dataset["compress_rate"])/len(lingua_dataset["compress_rate"])

0.9453304461232439

Generating

In [ ]:
pcrl_dataset = load_from_disk(r"/content/pcrl-old-dataset")
lingua1_dataset = load_from_disk(r"/content/llmlingua1-dataset")
lingua2_dataset = load_from_disk(r"/content/llmlingua2-dataset")

In [ ]:
generator_batch_size = 8
target_model = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",  # replace with "mps" to run on a Mac device
    batch_size=generator_batch_size
)

def generate_batch_predictions(batch, batch_size,run_name):
    start_time = time.time()
    predictions = []
    wandb.init(project="pcrl_exps", name=(run_name))
    for i in tqdm(range(0, len(batch), batch_size), desc="🧠 Generating answers"):
        sub_batch = batch[i:i+batch_size]

        chat_batch = [
            [{"role": "user", "content": f"""Concise answer strictly based on the provided context only. Do not generate responses beyond the given information. Do not think or infer. Respond concisely using only the context. \nContext: {sub_batch['context'][i]}\nQuestion: {sub_batch['question'][i]}\nAnswer:"""}]
            for i in range(len(sub_batch["question"]))
        ]

        outputs = target_model(
            chat_batch,
            return_full_text=False,
            max_new_tokens=50,
            min_new_tokens=1,
            batch_size=batch_size,
        )
        wandb.log({
            "VRAM Allocated (MB)": torch.cuda.memory_allocated() / (1024**2),
            "VRAM Reserved (MB)": torch.cuda.memory_reserved() / (1024**2)
        })

        predictions.extend([
            output[0]["generated_text"].removeprefix("assistant").strip().lower()
            for output in outputs
        ])
    total_time = time.time() - start_time
    print("\n===============\nTotal Time:",total_time,"\n===============\n")
    wandb.finish()
    return predictions

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
original_output = generate_batch_predictions(test_dataset,generator_batch_size,"Original Prompt final")
output_dataset = Dataset.from_dict({"answer" : original_output})
output_dataset.save_to_disk('org-answer-dataset')

pcrl_output = generate_batch_predictions(pcrl_dataset,generator_batch_size,"PCRL generate final")
output_dataset = Dataset.from_dict({"answer" : pcrl_output})
output_dataset.save_to_disk('pcrl-answer-dataset')

llmlingua_output = generate_batch_predictions(lingua1_dataset,generator_batch_size,"llmlingua-1 generate final")
output_dataset = Dataset.from_dict({"answer" : llmlingua_output})
output_dataset.save_to_disk('llmlingua1-answer-dataset')

llmlingua_output = generate_batch_predictions(lingua2_dataset,generator_batch_size,"llmlingua-2 generate final")
output_dataset = Dataset.from_dict({"answer" : llmlingua_output})
output_dataset.save_to_disk('llmlingua2-answer-dataset')

🧠 Generating answers: 100%|██████████| 1000/1000 [46:33<00:00,  2.79s/it]


Total Time: 2794.549904346466 



VRAM Allocated (MB),▁▁▂▂▂▂▂▂▂▄▄▄▄▄▄▄▄▄██████████████████████
VRAM Reserved (MB),▁▄▄▄▄▄▄▄▄▄██████████████████████████████
VRAM Allocated (MB),6011.86816
VRAM Reserved (MB),9138


Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

🧠 Generating answers: 100%|██████████| 1000/1000 [27:26<00:00,  1.65s/it]

...

Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

🧠 Generating answers: 100%|██████████| 1000/1000 [29:08<00:00,  1.75s/it]


Total Time: 1754.053317785263 



VRAM Allocated (MB),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
VRAM Reserved (MB),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
VRAM Allocated (MB),6011.86816
VRAM Reserved (MB),9140


Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

Answer Keep-rate

In [ ]:
pcrl_dataset = load_from_disk(r"/content/pcrl-dataset")
llmlingua1_dataset = load_from_disk(r"/content/llmlingua1-dataset")
llmlingua2_dataset = load_from_disk(r"/content/llmlingua2-dataset")

In [ ]:
print(sum(pcrl_dataset["pre_label"]),sum(pcrl_dataset["post_label"]))
print(sum(llmlingua1_dataset["pre_label"]),sum(llmlingua1_dataset["post_label"]))
print(sum(llmlingua2_dataset["pre_label"]),sum(llmlingua2_dataset["post_label"]))

4312 2464
4312 2000
4312 1000


In [ ]:
pcrl_output = load_from_disk(r"/content/pcrl-answer-dataset")["answer"]
llmlingua1_output = load_from_disk(r"/content/llmlingua1-answer-dataset")["answer"]
llmlingua2_output = load_from_disk(r"/content/llmlingua2-answer-dataset")["answer"]
original_output = load_from_disk(r"/content/org-answer-dataset")["answer"]

llama 3.2 3b instruct

In [ ]:
pcrl_dataset = load_from_disk(r"/content/pcrl-dataset")
lingua1_dataset = load_from_disk(r"/content/llmlingua1-dataset")
lingua2_dataset = load_from_disk(r"/content/llmlingua2-dataset")

In [ ]:
generator_batch_size = 8
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct",padding_side='left')
llama_tokenizer.pad_token = llama_tokenizer.eos_token

target_model = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-3B-Instruct",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",  # replace with "mps" to run on a Mac device
    batch_size=generator_batch_size,
    tokenizer=llama_tokenizer,
)


def generate_batch_predictions(batch, batch_size,run_name):
    start_time = time.time()
    predictions = []
    wandb.init(project="pcrl_exps", name=(run_name+" llama3.2"))
    for i in tqdm(range(0, len(batch), batch_size), desc="🧠 Generating answers"):
        sub_batch = batch[i:i+batch_size]

        chat_batch = [
            [{"role": "user", "content": f"""Concise answer strictly based on the provided context only. Do not generate responses beyond the given information. Do not think or infer. Respond concisely using only the context. \nContext: {sub_batch['context'][i]}\nQuestion: {sub_batch['question'][i]}\nAnswer:"""}]
            for i in range(len(sub_batch["question"]))
        ]

        outputs = target_model(
            chat_batch,
            return_full_text=False,
            max_new_tokens=50,
            min_new_tokens=1,
            batch_size=batch_size,
        )
        wandb.log({
            "VRAM Allocated (MB)": torch.cuda.memory_allocated() / (1024**2),
            "VRAM Reserved (MB)": torch.cuda.memory_reserved() / (1024**2)
        })

        predictions.extend([
            output[0]["generated_text"].removeprefix("assistant").strip().lower()
            for output in outputs
        ])
    total_time = time.time() - start_time
    print("\n===============\nTotal Time:",total_time,"\n===============\n")
    wandb.finish()
    return predictions

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
original_output = generate_batch_predictions(test_dataset,8,"Original Prompt final")
output_dataset = Dataset.from_dict({"answer" : original_output})
output_dataset.save_to_disk('org-answer-llama-dataset')

pcrl_output = generate_batch_predictions(pcrl_dataset,8,"PCRL generate final")
output_dataset = Dataset.from_dict({"answer" : pcrl_output})
output_dataset.save_to_disk('pcrl-answer-llama-dataset')

llmlingua_output = generate_batch_predictions(lingua1_dataset,8,"llmlingua-1 generate final")
output_dataset = Dataset.from_dict({"answer" : llmlingua_output})
output_dataset.save_to_disk('llmlingua1-answer-llama-dataset')

llmlingua_output = generate_batch_predictions(lingua2_dataset,8,"llmlingua-2 generate final")
output_dataset = Dataset.from_dict({"answer" : llmlingua_output})
output_dataset.save_to_disk('llmlingua2-answer-llama-dataset')

🧠 Generating answers: 100%|██████████| 1000/1000 [56:39<00:00,  3.40s/it]


Total Time: 3400.8185880184174 



VRAM Allocated (MB),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
VRAM Reserved (MB),▁▁▁▁▁▃▃▃▃▃▄▄▄▄▄▄▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆█████
VRAM Allocated (MB),6136.45947
VRAM Reserved (MB),10116


Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

🧠 Generating answers: 100%|██████████| 1000/1000 [20:21<00:00,  1.22s/it]

...

Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

🧠 Generating answers: 100%|██████████| 1000/1000 [21:34<00:00,  1.29s/it]


Total Time: 1295.640191078186 



VRAM Allocated (MB),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
VRAM Reserved (MB),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
VRAM Allocated (MB),6136.45947
VRAM Reserved (MB),10120


Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

By Part


In [ ]:
import collections
import logging
import regex
import string
import unicodedata
from functools import partial
from multiprocessing import Pool as ProcessPool
from typing import Tuple, List, Dict
import numpy as np
from datasets import load_from_disk

from collections import Counter

"""
Evaluation code from DPR: https://github.com/facebookresearch/DPR
"""

class SimpleTokenizer(object):
    ALPHA_NUM = r'[\p{L}\p{N}\p{M}]+'
    NON_WS = r'[^\p{Z}\p{C}]'

    def __init__(self):
        """
        Args:
            annotators: None or empty set (only tokenizes).
        """
        self._regexp = regex.compile(
            '(%s)|(%s)' % (self.ALPHA_NUM, self.NON_WS),
            flags=regex.IGNORECASE + regex.UNICODE + regex.MULTILINE
        )

    def tokenize(self, text, uncased=False):
        matches = [m for m in self._regexp.finditer(text)]
        if uncased:
            tokens = [m.group().lower() for m in matches]
        else:
            tokens = [m.group() for m in matches]
        return tokens

logger = logging.getLogger(__name__)

QAMatchStats = collections.namedtuple('QAMatchStats', ['top_k_hits', 'questions_doc_hits'])

def calculate_matches(data: List, workers_num: int):
    """
    Evaluates answers presence in the set of documents. This function is supposed to be used with a large collection of
    documents and results. It internally forks multiple sub-processes for evaluation and then merges results
    :param all_docs: dictionary of the entire documents database. doc_id -> (doc_text, title)
    :param answers: list of answers's list. One list per question
    :param closest_docs: document ids of the top results along with their scores
    :param workers_num: amount of parallel threads to process data
    :param match_type: type of answer matching. Refer to has_answer code for available options
    :return: matching information tuple.
    top_k_hits - a list where the index is the amount of top documents retrieved and the value is the total amount of
    valid matches across an entire dataset.
    questions_doc_hits - more detailed info with answer matches for every question and every retrieved document
    """

    logger.info('Matching answers in top docs...')

    tokenizer = SimpleTokenizer()
    get_score_partial = partial(check_answer, tokenizer=tokenizer)

    processes = ProcessPool(processes=workers_num)
    scores = processes.map(get_score_partial, data)

    logger.info('Per question validation results len=%d', len(scores))

    n_docs = len(data[0]['ctxs'])
    top_k_hits = [0] * n_docs
    for question_hits in scores:
        best_hit = next((i for i, x in enumerate(question_hits) if x), None)
        if best_hit is not None:
            top_k_hits[best_hit:] = [v + 1 for v in top_k_hits[best_hit:]]

    return QAMatchStats(top_k_hits, scores)

def check_answer(example, tokenizer) -> List[bool]:
    """Search through all the top docs to see if they have any of the answers."""
    answers = example['answers']
    ctxs = example['ctxs']

    hits = []

    for i, doc in enumerate(ctxs):
        text = doc['text']

        if text is None:  # cannot find the document for some reason
            logger.warning("no doc in db")
            hits.append(False)
            continue

        hits.append(has_answer(answers, text, tokenizer))

    return hits

def has_answer(answers, text, tokenizer) -> bool:
    """Check if a document contains an answer string."""
    text = _normalize(text)
    text = tokenizer.tokenize(text, uncased=True)

    for answer in answers:
        answer = _normalize(answer)
        answer = tokenizer.tokenize(answer, uncased=True)
        for i in range(0, len(text) - len(answer) + 1):
            if answer == text[i: i + len(answer)]:
                return True
    return False

#################################################
########        READER EVALUATION        ########
#################################################

def _normalize(text):
    return unicodedata.normalize('NFD', text)

#Normalization from SQuAD evaluation script https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/
def normalize_answer(s):
    def remove_articles(text):
        return regex.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def ems(prediction, ground_truths):
    return max([exact_match_score(prediction, gt) for gt in ground_truths])

####################################################
########        RETRIEVER EVALUATION        ########
####################################################

def eval_batch(scores, inversions, avg_topk, idx_topk):
    for k, s in enumerate(scores):
        s = s.cpu().numpy()
        sorted_idx = np.argsort(-s)
        score(sorted_idx, inversions, avg_topk, idx_topk)

def count_inversions(arr):
    inv_count = 0
    lenarr = len(arr)
    for i in range(lenarr):
        for j in range(i + 1, lenarr):
            if (arr[i] > arr[j]):
                inv_count += 1
    return inv_count

def score(x, inversions, avg_topk, idx_topk):
    x = np.array(x)
    inversions.append(count_inversions(x))
    for k in avg_topk:
        # ratio of passages in the predicted top-k that are
        # also in the topk given by gold score
        avg_pred_topk = (x[:k]<k).mean()
        avg_topk[k].append(avg_pred_topk)
    for k in idx_topk:
        below_k = (x<k)
        # number of passages required to obtain all passages from gold top-k
        idx_gold_topk = len(x) - np.argmax(below_k[::-1])
        idx_topk[k].append(idx_gold_topk)

"""
our evaluation code
"""

def f1_score(prediction, ground_truth):
    normalized_prediction = normalize_answer(prediction)
    normalized_ground_truth = normalize_answer(ground_truth)

    ZERO_METRIC = (0, 0, 0, 0)

    if normalized_prediction in ['yes', 'no', 'noanswer'] and normalized_prediction != normalized_ground_truth:
        return ZERO_METRIC
    if normalized_ground_truth in ['yes', 'no', 'noanswer'] and normalized_prediction != normalized_ground_truth:
        return ZERO_METRIC

    accuracy = 1.0 if normalized_ground_truth in normalized_prediction else 0.0

    prediction_tokens = normalized_prediction.split()
    ground_truth_tokens = normalized_ground_truth.split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return ZERO_METRIC
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return (f1, precision, recall, accuracy)

def f1_scores(prediction, ground_truths):
    max_f1 = max_precision = max_recall = max_acc = 0

    for gt in ground_truths:
        score = f1_score(prediction, gt)
        max_f1 = max(max_f1, score[0])  # Accessing F1 score from the tuple
        max_precision = max(max_precision, score[1])  # Accessing precision from the tuple
        max_recall = max(max_recall, score[2])  # Accessing recall from the tuple
        max_acc = max(max_acc, score[3])

    return max_f1, max_precision, max_recall, max_acc

def evaluate_QA(results, ans_key, predict_key):
    """
    EVALUATION
    """
    metrics = {'em': 0, 'f1': 0, 'prec': 0, 'recall': 0, 'acc': 0}

    em_for_task = ems
    f1_for_task = f1_scores

    for result in results:
        prediction = result[predict_key]
        gold = result[ans_key]

        em = em_for_task(prediction, gold)
        f1, prec, recall, acc = f1_for_task(prediction, gold)

        metrics['em'] += float(em)
        metrics['f1'] += f1
        metrics['prec'] += prec
        metrics['recall'] += recall
        metrics['acc'] += acc

        result['metrics'] = {'em': float(em), 'f1': f1, 'prec': prec, 'recall': recall, 'acc': acc}

    for k in metrics.keys():
        metrics[k] /= len(results)

    return metrics

def gather_result(data):
    results = []
    for gold,predict in zip(test_dataset,data) :
        results.append({
                "prediction": predict,
                "gold": gold["answer"],
        })
    return results


In [ ]:
pcrl_output = load_from_disk(r"/content/pcrl-answer-dataset")["answer"]
llmlingua1_output = load_from_disk(r"/content/llmlingua1-answer-dataset")["answer"]
llmlingua2_output = load_from_disk(r"/content/llmlingua2-answer-dataset")["answer"]
original_output = load_from_disk(r"/content/org-answer-dataset")["answer"]

In [ ]:
#All dataset
print("Original: ",evaluate_QA(gather_result(original_output), "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output), "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output), "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output), "gold", "prediction"))

Original:  {'em': 0.2725, 'f1': 0.3931834889419135, 'prec': 0.4491005902645693, 'recall': 0.47240258245857386, 'acc': 0.40675}
PCRL compress:  {'em': 0.24425, 'f1': 0.3379167798958191, 'prec': 0.3972670766314195, 'recall': 0.3876451459390014, 'acc': 0.327875}
LLMLingua1 compress:  {'em': 0.105125, 'f1': 0.18665470959406513, 'prec': 0.23711297348712834, 'recall': 0.25335962534035067, 'acc': 0.18775}
LLMLingua2 compress:  {'em': 0.124, 'f1': 0.20721959302662937, 'prec': 0.26379634383324213, 'recall': 0.2695618075934916, 'acc': 0.207375}


In [ ]:
#TQA
print("Original: ",evaluate_QA(gather_result(original_output)[:2000], "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output)[:2000], "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output)[:2000], "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output)[:2000], "gold", "prediction"))

Original:  {'em': 0.513, 'f1': 0.611263490012808, 'prec': 0.5925869896609525, 'recall': 0.6994938061938059, 'acc': 0.663}
PCRL compress:  {'em': 0.511, 'f1': 0.5965531684517755, 'prec': 0.5840904247713073, 'recall': 0.650722619047619, 'acc': 0.6185}
LLMLingua1 compress:  {'em': 0.24, 'f1': 0.30928694440059945, 'prec': 0.2981013126450434, 'recall': 0.37389954212454235, 'acc': 0.311}
LLMLingua2 compress:  {'em': 0.26, 'f1': 0.33422816484836165, 'prec': 0.3312957121503369, 'recall': 0.37907916666666686, 'acc': 0.3185}


In [ ]:
#NQA
print("Original: ",evaluate_QA(gather_result(original_output)[2000:4000], "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output)[2000:4000], "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output)[2000:4000], "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output)[2000:4000], "gold", "prediction"))

Original:  {'em': 0.2755, 'f1': 0.4069183608689977, 'prec': 0.40506113107260205, 'recall': 0.48673660922410933, 'acc': 0.4075}
PCRL compress:  {'em': 0.23, 'f1': 0.3421540168649463, 'prec': 0.34797567432761645, 'recall': 0.3905703449328451, 'acc': 0.311}
LLMLingua1 compress:  {'em': 0.094, 'f1': 0.1656676809166115, 'prec': 0.16414629826936597, 'recall': 0.2232534923409921, 'acc': 0.1565}
LLMLingua2 compress:  {'em': 0.1065, 'f1': 0.18459554572718415, 'prec': 0.18794936440044366, 'recall': 0.238579827117327, 'acc': 0.1695}


In [ ]:
#HQA
print("Original: ",evaluate_QA(gather_result(original_output)[4000:6000], "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output)[4000:6000], "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output)[4000:6000], "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output)[4000:6000], "gold", "prediction"))

Original:  {'em': 0.138, 'f1': 0.32421679796074454, 'prec': 0.5784416849948398, 'recall': 0.41189300965447956, 'acc': 0.3005}
PCRL compress:  {'em': 0.13, 'f1': 0.25853949641456525, 'prec': 0.5146398193289565, 'recall': 0.27703920707712837, 'acc': 0.1815}
LLMLingua1 compress:  {'em': 0.048, 'f1': 0.16418409068571363, 'prec': 0.4007719381900106, 'recall': 0.1816491573720628, 'acc': 0.08}
LLMLingua2 compress:  {'em': 0.0485, 'f1': 0.1589346261369634, 'prec': 0.4077713054872086, 'recall': 0.17182730585404818, 'acc': 0.0795}


In [ ]:
#2wiki
print("Original: ",evaluate_QA(gather_result(original_output)[6000:], "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output)[6000:], "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output)[6000:], "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output)[6000:], "gold", "prediction"))

Original:  {'em': 0.1635, 'f1': 0.2303353069251112, 'prec': 0.22031255532988156, 'recall': 0.29148690476190486, 'acc': 0.256}
PCRL compress:  {'em': 0.106, 'f1': 0.15442043785199602, 'prec': 0.1423623880977982, 'recall': 0.23224841269841245, 'acc': 0.2005}
LLMLingua1 compress:  {'em': 0.0385, 'f1': 0.107480122373344, 'prec': 0.08543234484409083, 'recall': 0.23463630952380943, 'acc': 0.2035}
LLMLingua2 compress:  {'em': 0.081, 'f1': 0.1511200353940151, 'prec': 0.12816899329497844, 'recall': 0.28876093073593073, 'acc': 0.262}


llama3.2 3b instruct

In [ ]:
pcrl_output = load_from_disk(r"/content/pcrl-answer-llama-dataset")["answer"]
llmlingua1_output = load_from_disk(r"/content/llmlingua1-answer-llama-dataset")["answer"]
llmlingua2_output = load_from_disk(r"/content/llmlingua2-answer-llama-dataset")["answer"]
original_output = load_from_disk(r"/content/org-answer-llama-dataset")["answer"]

In [ ]:
#All dataset
print("Original: ",evaluate_QA(gather_result(original_output), "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output), "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output), "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output), "gold", "prediction"))

Original:  {'em': 0.187375, 'f1': 0.32547589035328583, 'prec': 0.37021133643646303, 'recall': 0.46218475139786447, 'acc': 0.39125}
PCRL compress:  {'em': 0.2315, 'f1': 0.3273034692862757, 'prec': 0.3842172304525247, 'recall': 0.38410565862577417, 'acc': 0.327875}
LLMLingua1 compress:  {'em': 0.090375, 'f1': 0.1798751381882916, 'prec': 0.22699702882830877, 'recall': 0.26817992202719515, 'acc': 0.20075}
LLMLingua2 compress:  {'em': 0.111375, 'f1': 0.19849395678413165, 'prec': 0.2510347944125586, 'recall': 0.27237082177807187, 'acc': 0.21425}


In [ ]:
#TQA
print("Original: ",evaluate_QA(gather_result(original_output)[:2000], "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output)[:2000], "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output)[:2000], "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output)[:2000], "gold", "prediction"))

Original:  {'em': 0.4455, 'f1': 0.5633920152803114, 'prec': 0.5361520640055373, 'recall': 0.701151406926407, 'acc': 0.656}
PCRL compress:  {'em': 0.54, 'f1': 0.6239971218027536, 'prec': 0.6129546240060254, 'recall': 0.6778809523809523, 'acc': 0.647}
LLMLingua1 compress:  {'em': 0.235, 'f1': 0.315748743521705, 'prec': 0.30126495652571844, 'recall': 0.4016170995671001, 'acc': 0.327}
LLMLingua2 compress:  {'em': 0.2965, 'f1': 0.37080303046730223, 'prec': 0.36805237743804703, 'recall': 0.4241004148629154, 'acc': 0.369}


In [ ]:
#NQA
print("Original: ",evaluate_QA(gather_result(original_output)[2000:4000], "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output)[2000:4000], "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output)[2000:4000], "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output)[2000:4000], "gold", "prediction"))

Original:  {'em': 0.181, 'f1': 0.3109988451518759, 'prec': 0.300296156098669, 'recall': 0.4296911061161062, 'acc': 0.348}
PCRL compress:  {'em': 0.19, 'f1': 0.30885966308101437, 'prec': 0.3163300696988797, 'recall': 0.361517687867688, 'acc': 0.28}
LLMLingua1 compress:  {'em': 0.064, 'f1': 0.13905062415620675, 'prec': 0.1337542726766258, 'recall': 0.20902155344655327, 'acc': 0.1385}
LLMLingua2 compress:  {'em': 0.077, 'f1': 0.16072006496092808, 'prec': 0.1647661341168225, 'recall': 0.2263060356310354, 'acc': 0.157}


In [ ]:
#HQA
print("Original: ",evaluate_QA(gather_result(original_output)[4000:6000], "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output)[4000:6000], "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output)[4000:6000], "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output)[4000:6000], "gold", "prediction"))

Original:  {'em': 0.095, 'f1': 0.2903957619637159, 'prec': 0.5393171984557916, 'recall': 0.40293796080291205, 'acc': 0.2855}
PCRL compress:  {'em': 0.142, 'f1': 0.2579058596643065, 'prec': 0.5060349714781108, 'recall': 0.2775130635185252, 'acc': 0.191}
LLMLingua1 compress:  {'em': 0.05, 'f1': 0.1627132641195415, 'prec': 0.40262321222889913, 'recall': 0.1854290509681473, 'acc': 0.0905}
LLMLingua2 compress:  {'em': 0.0475, 'f1': 0.15284268242537413, 'prec': 0.3895513594316964, 'recall': 0.1637304080469116, 'acc': 0.076}


In [ ]:
#2wiki
print("Original: ",evaluate_QA(gather_result(original_output)[6000:], "gold", "prediction"))
print("PCRL compress: ",evaluate_QA(gather_result(pcrl_output)[6000:], "gold", "prediction"))
print("LLMLingua1 compress: ",evaluate_QA(gather_result(llmlingua1_output)[6000:], "gold", "prediction"))
print("LLMLingua2 compress: ",evaluate_QA(gather_result(llmlingua2_output)[6000:], "gold", "prediction"))

Original:  {'em': 0.028, 'f1': 0.1371169390172342, 'prec': 0.105079927185863, 'recall': 0.3149585317460318, 'acc': 0.2755}
PCRL compress:  {'em': 0.054, 'f1': 0.11845123259702671, 'prec': 0.10154925662708728, 'recall': 0.2195109307359305, 'acc': 0.1935}
LLMLingua1 compress:  {'em': 0.0125, 'f1': 0.10198792095572068, 'prec': 0.07034567388198905, 'recall': 0.27665198412698394, 'acc': 0.247}
LLMLingua2 compress:  {'em': 0.0245, 'f1': 0.10961004928292906, 'prec': 0.08176930666366547, 'recall': 0.27534642857142855, 'acc': 0.255}
